<a href="https://colab.research.google.com/github/jamesbaskerville/colabs/blob/main/WikipediaCrawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Goal is to crawl English Wikipedia and get a set of all wikipedia pages within n hops of https://en.wikipedia.org/wiki/Computer_science


#### Scratchpad

- will need to build adjacency graph or at least figure out how to queue links
- single hop needs how to extract links
- this probably looks like beautiful soup for HTML parsing
  - simple = find all anchors, filter by ones that start with en.wikipedia.com/wiki
- parsers should probably be
- where is the content stored? is there a text version of wikipedia

#### API
https://en.wikipedia.org/w/api.php?action=parse&page=Computer%20science&prop=links&format=json
wikipedia has an api that can easily pull links from an article. Maybe good place to start to avoid HTML parsing for now. Just need JSON parsing.

In [2]:
!pip install aiohttp
!pip install sentence-transformers

In [1]:
import asyncio
import aiohttp
import time
import logging

class RateLimiter:
    def __init__(self, requests_per_second):
        self.requests_per_second = requests_per_second
        self.last_request_time = 0

    async def wait_for_rate_limit(self):
        current_time = time.time()
        time_since_last_request = current_time - self.last_request_time
        if time_since_last_request < 1 / self.requests_per_second:
            await asyncio.sleep(1 / self.requests_per_second - time_since_last_request)
        self.last_request_time = time.time()

class WikipediaClient:
  def __init__(self, rate_limit=5):
    # include rate limiting
    self.rate_limiter = RateLimiter(5)

  async def fetch_page(self, page: str, prop: str):
    async with aiohttp.ClientSession() as session:
      await self.rate_limiter.wait_for_rate_limit()
      if prop == 'content':
        url = f'https://en.wikipedia.org/w/api.php?action=query&prop=extracts&format=json&explaintext&titles={page}'
      else:
        url = f'https://en.wikipedia.org/w/api.php?action=parse&page={page}&props=links&format=json'

      logging.debug(f"Fetching {url}")
      async with session.get(url) as response:
        response = await response.json()

      if prop == 'content':
        logging.debug(f"Fetched content for {url}")
        try:
          return [x['extract'] for x in response['query']['pages'].values()]
        except:
          print(f"Response: {response}")
          return ""
      else:
        logging.debug(f"Fetched links for {url}")
        logging.debug(f"Response: {response}")
        return [x["*"] for x in filter(lambda x: x['ns'] == 0, response['parse']['links'])]

  async def get_adjacent_pages(self, page: str):
    page_names = await self.fetch_page(page, 'links')
    return page_names

  async def get_page_content(self, page: str):
    return await self.fetch_page(page, 'content')

In [27]:
from sentence_transformers import SentenceTransformer
from sentence_transformers import util


class SentenceEmbedder:
  def __init__(self):
    self.model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

  def embed(self, text):
    return self.model.encode(text, convert_to_tensor=True)

  # 0 to 1 with 0 being most similar
  def similarity(self, embedding1, embedding2):
    cos_sim = util.cos_sim(embedding1, embedding2).item()
    # normalize
    return (-1 * cos_sim + 1) / 2

test_embedder = SentenceEmbedder()
test_embedder.similarity(test_embedder.embed("hello how are you"), test_embedder.embed("i really dislike you"))

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


0.40385255217552185

In [63]:
import traceback

# Naive way to search for target = BFS
# Idea = use semantic comparison somehow to word2vec or otherwise
# compare current & target and use a priority queue BFS
# should also store path and update in in maybe a dictionary instead
# of a visited set; e.g. store shortest path to a node in the dict value
class WikipediaCrawler:
  def __init__(self, start_page: str, target='Kevin Bacon', max_concurrent=5, rate_limit=5, max_depth=5, strategy="depth"):
    self.start_page = start_page
    self.target = target
    self.queue = asyncio.PriorityQueue()
    self.visited = {}
    self.embeddings = {}
    self.errored = set()
    self.max_concurrent = max_concurrent
    self.rate_limit = rate_limit
    self.max_depth = max_depth
    self.stop_event = asyncio.Event()
    self.strategy = strategy
    self.embedder = SentenceEmbedder()
    self.client = WikipediaClient(rate_limit)
    self.workers = []


  async def priority_score(self, page, prev):
    if self.strategy == "depth":
      return len(self.visited[prev]) if prev in self.visited else 0
    elif self.strategy == "embedding":
      if self.target not in self.embeddings:
        target_content = await self.client.get_page_content(self.target)
        self.embeddings[self.target] = self.embedder.embed(target_content)

      if page not in self.embeddings:
        page_content = await self.client.get_page_content(page)
        if not page_content:
          return 1.0
        self.embeddings[page] = self.embedder.embed(page_content)
      result = self.embedder.similarity(self.embeddings[page], self.embeddings[self.target])
      return result
    else:
      return 1.0


  async def crawl(self):
    await self.queue.put((0, self.start_page, None)) # depth/priority, page, prev
    self.workers = [asyncio.create_task(self.worker(f'worker{i}')) for i in range(self.max_concurrent)]

    try:
      await asyncio.gather(*self.workers)
      print('Done with workers')
      # for worker in self.workers:
      #   worker.cancel()
    except asyncio.CancelledError:
        print("Tasks cancelled.")
    finally:
        self.stop_event.set()  # Signal workers to stop
        # runtime.unassign()  # Terminate Colab runtime

    if self.target in self.visited:
      print(f"Found target page {self.target} in {len(self.visited[self.target])} hops")
      print(f"Path: {self.visited[self.target]}")
      print(f"Total pages visited: {len(self.visited)}")
      return True
    else:
      print(f"Did not find target page {self.target}")
      print(f"Total pages visited: {len(self.visited)}")
      return False

  async def worker(self, workerName, prio_strategy="depth"):
    print(f"Starting {workerName}")

    while not self.stop_event.is_set():
      prio_score, page, prev = await self.queue.get()
      print(f"Queue length: {self.queue.qsize()}; processing {page} with score {prio_score}")

      depth = len(self.visited[prev]) if prev else 0
      if depth > self.max_depth:
        self.queue.task_done()
        return

      # update to shorter path if possible
      if page in self.visited and len(self.visited[prev]) + 1 < len(self.visited[page]):
        self.visited[page] = self.visited[prev] + [prev]
        self.queue.task_done()
        continue

      if page not in self.visited:
        if not prev:
          self.visited[page] = [page]
        else:
          self.visited[page] = self.visited[prev] + [page]

        if len(self.visited) % 200 == 0:
          print(f"Processed {len(self.visited)} pages, currently at depth {depth}, queue length {self.queue.qsize()}")

        try:
          adjacent_pages = await self.client.get_adjacent_pages(page)

          # for the embeddings approach this is probably too heavy -- maybe should just sample 10 or 20 and move on
          for adjacent_page in adjacent_pages:
            if self.stop_event.is_set():
              print(f"Stopping {workerName}")
              self.queue.task_done()
              return

            # Success!
            if adjacent_page == self.target:
              self.visited[self.target] = self.visited[prev] + [page, adjacent_page]
              self.queue.task_done()
              print(f"Found target with path: {self.visited[self.target]}")
              self.stop_event.set()

            elif adjacent_page not in self.visited and depth + 1 <= self.max_depth:
              new_prio_score = await self.priority_score(adjacent_page, prev)
              print(f"Adding {adjacent_page} with priority {new_prio_score}")
              await self.queue.put((new_prio_score, adjacent_page, page))
            else:
              # logging.debug(f"Skipping {adjacent_page}")
              pass
        except Exception as e:
          print(f"Error processing {page}: {e}")
          traceback.print_exc()
          self.errored.add(page)
          # retry once
          if page not in self.errored:
            print(f"Retrying {page}")
            await self.queue.put((prio_score, page, prev))

      self.queue.task_done()
    print(f"{workerName} exiting gracefully.")
    return

In [64]:
# logging.basicConfig(level=logging.DEBUG)
crawler = WikipediaCrawler(
    'Computer science',
    target="Singapore",
    rate_limit=5,
    max_depth=2,
    strategy="embedding"
  )
await crawler.crawl()
# getting rate limited, so should slow it down

Starting worker0
Queue length: 0; processing Computer science with score 0
Starting worker1
Starting worker2
Starting worker3
Starting worker4
Adding Computer science (disambiguation) with priority 0.4824706446379423
Queue length: 0; processing Computer science (disambiguation) with score 0.4824706446379423
Adding History of computer science with priority 0.45875096693634987
Queue length: 0; processing History of computer science with score 0.45875096693634987
Adding History of computing with priority 0.4832000471651554
Queue length: 0; processing History of computing with score 0.4832000471651554
Adding History of informatics with priority 0.5356547497212887
Queue length: 0; processing History of informatics with score 0.5356547497212887
Adding Computer Science (UIL) with priority 0.48631735146045685
Adding Informatics with priority 0.3937124013900757
Adding Informatics with priority 0.3937124013900757
Queue length: 2; processing Informatics with score 0.3937124013900757
Adding Comput

False

In [25]:
crawler.workers

[<Task cancelled name='Task-2' coro=<WikipediaCrawler.worker() done, defined at <ipython-input-12-4c1d5a140ab8>:67>>,
 <Task cancelled name='Task-3' coro=<WikipediaCrawler.worker() done, defined at <ipython-input-12-4c1d5a140ab8>:67>>,
 <Task cancelled name='Task-4' coro=<WikipediaCrawler.worker() done, defined at <ipython-input-12-4c1d5a140ab8>:67>>,
 <Task cancelled name='Task-5' coro=<WikipediaCrawler.worker() done, defined at <ipython-input-12-4c1d5a140ab8>:67>>,
 <Task cancelled name='Task-6' coro=<WikipediaCrawler.worker() done, defined at <ipython-input-12-4c1d5a140ab8>:67>>]

In [30]:
test_embedder.similarity(crawler.embeddings['Marvin Minsky'], crawler.embeddings['Harvard University'])

0.34969037771224976

In [34]:
client = WikipediaClient()
await client.find_adjacent_pages('Distance')

['Distance (disambiguation)',
 'Euclidean distance',
 'Distance measurement',
 'Geographic distance',
 'Geodesic',
 'Distance measure',
 'Statistical distance',
 'Distance (graph theory)',
 'Metric space',
 'Length',
 'Displacement (vector)',
 'Signed distance function',
 'Absement',
 'Absolute difference',
 'Acceleration',
 'Affine subspace',
 'Altitude',
 'Ant',
 'Arc length',
 'As the crow flies',
 'Astronomical system of units',
 'Bacon number',
 'Boundary (topology)',
 'Bregman divergence',
 'Cartesian coordinate system',
 'Center of mass',
 'Chebyshev distance',
 'Chessboard',
 'CiteSeerX (identifier)',
 'Classical physics',
 'Closed curve',
 'Closeness (mathematics)',
 'Coding theory',
 'Color difference',
 'Compact space',
 'Computer science',
 'Cosmic distance ladder',
 'Cosmology',
 'Derivative',
 "Dijkstra's algorithm",
 'Direction (geometry, geography)',
 'Displacement (geometry)',
 'Distance from a point to a line',
 'Distance from a point to a plane',
 'Distance geometry 